# CRRT Mortality Prediction
## Extraction of Patient Cohort and Feature Vectors
### Christopher V. Cosgriff, David Sasson, Colby Wilkinson, Kanhua Yin

The goal of this notebook is to form a preliminary cohort, and to extract the vital and lab features for our cohort.

## Step 0: Environment Setup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import psycopg2

# postgres envrionment setup; specific to me
sqluser = 'cosgriffc'
dbname = 'mimic'
schema_name = 'mimiciii'
host = 'localhost'

query_schema = 'SET search_path TO ' + schema_name + ';'

# connect to the database
con = psycopg2.connect(dbname = dbname, user = sqluser, host = host)

# plotting stuffs
# "Tableau 20" colors as RGB.   
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]  
  
# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.)

marker = ['v','o','d','^','s','>','+']
ls = ['-','-','-','-','-','s','--','--']

# configure jupyter for using matplotlib on rMBP
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

## Step 1: Load base cohort 

Before we can run queries we need to generate the requisite materialized views. They are:

1. ICU stay detail (`icustay_detail`)
2. Vitals only CE pull (`vitals`)
3. Labs only LE pull (`labs`)
4. OASIS (`oasis`)
5. OASIS dependencies: `uofirsday`, `vitalsfirstday`, `gcsfirstday`, `ventfirstday`
6. Elixhauser score `elixhauser-ahrq-v37-no-drg-all-icd` and `elixhauser_ahrq_no_drg_all_icd_score`
7. CRRT Durations `crrt-durations`

In [2]:
query = query_schema + '''
WITH icu_death AS
(
SELECT ie.icustay_id
    , CASE
        WHEN adm.deathtime BETWEEN ie.intime and ie.outtime
            THEN 1
        -- sometimes there are typographical errors in the death date, so check before intime
        WHEN adm.deathtime <= ie.intime
            THEN 1
        WHEN adm.dischtime <= ie.outtime
            AND adm.discharge_location = 'DEAD/EXPIRED'
            THEN 1
        ELSE 0
        END AS icustay_expire_flag
    , CASE
        WHEN adm.deathtime <= (adm.dischtime + interval '90' day) THEN 1
        ELSE 0
        END AS ninety_expire_flag
FROM icustays ie
INNER JOIN admissions adm
ON ie.hadm_id = adm.hadm_id
),
oasis AS
(
    SELECT icustay_id, oasis
    FROM oasis
),
elixhauser AS
(
    SELECT hadm_id, elixhauser_vanwalraven FROM elixhauser_ahrq_score
)
, crrt AS
(
    SELECT i.hadm_id, SUM(c.duration_hours) as crrt_duration 
    FROM crrtdurations c
    INNER JOIN icustays i
    ON c.icustay_id = i.icustay_id
    GROUP BY i.hadm_id
)

SELECT id.hadm_id, id.icustay_id, id.admission_age, id.gender, id.ethnicity, 
       id.intime, id.outtime, id.los_icu, id.icustay_seq, elix.elixhauser_vanwalraven, oa.oasis
       , CASE WHEN id.los_icu < 0.167 THEN 1
       ELSE 0
       END AS exclude_los
       , CASE WHEN id.admission_age < 16 THEN 1
       ELSE 0
       END AS exclude_age
       , idth.ninety_expire_flag
FROM icustay_detail id
INNER JOIN icu_death idth
ON id.icustay_id = idth.icustay_id
INNER JOIN oasis oa
ON id.icustay_id = oa.icustay_id
INNER JOIN elixhauser elix
ON id.hadm_id = elix.hadm_id
INNER JOIN crrt
ON id.hadm_id = crrt.hadm_id;
'''
    
icu_base = pd.read_sql_query(query, con)
icu_base.head()

,hadm_id,icustay_id,admission_age,gender,ethnicity,intime,outtime,los_icu,icustay_seq,elixhauser_vanwalraven,oasis,exclude_los,exclude_age,ninety_expire_flag
0,100053,217069,56.6149,M,WHITE,2124-07-14 03:20:29,2124-07-19 01:54:09,4.9400,1,24,48,0,0,1
1,100095,220320,84.2335,M,UNKNOWN/NOT SPECIFIED,2108-09-28 20:01:23,2108-10-05 10:46:47,6.6149,1,17,28,0,0,1
2,100098,216749,71.5343,F,WHITE,2108-04-04 15:26:39,2108-05-08 14:47:19,33.9727,1,17,33,0,0,1
3,100108,273316,60.0889,M,UNKNOWN/NOT SPECIFIED,2143-04-29 23:28:39,2143-05-12 18:19:27,12.7853,1,6,49,0,0,1
4,100364,285421,85.7991,M,WHITE,2124-02-03 11:03:35,2124-03-08 14:52:28,34.1589,1,23,47,0,0,0


## Step 2: Apply Exclusion Criteria

In the previous pull I added flages for exclusion. They are:
1. Age <16 (subject level)
2. Length of stay in the ICU <4h (icustay level)

In [3]:
# First just drop any patient with an 
icu_cohort = icu_base[icu_base.exclude_age == 0]

# Exclude hadm_id for los_exclude or died during first ICU stay
exclude_hadm_ids = icu_cohort.hadm_id[(icu_cohort.icustay_seq >= 3) | (np.isnan(icu_cohort.los_icu))]

# Drop variables no longer needed
icu_cohort = icu_cohort.drop(['exclude_age', 'exclude_los'], axis = 1)

In [4]:
icu_cohort.describe().T

,count,mean,std,min,25%,50%,75%,max
hadm_id,1301.0,148959.273636,29192.003474,100053.0000,123404.0000,148860.0000,174824.0000,199919.0000
icustay_id,1301.0,249784.129900,28869.475428,200065.0000,224945.0000,250300.0000,274326.0000,299994.0000
admission_age,1301.0,63.889437,27.018418,20.2686,51.9982,62.4700,73.3998,302.2868
los_icu,1301.0,13.518671,14.276263,0.0012,3.9319,9.0294,17.8257,153.9280
icustay_seq,1301.0,1.280553,0.644860,1.0000,1.0000,1.0000,1.0000,6.0000
elixhauser_vanwalraven,1301.0,14.716372,7.811941,-7.0000,9.0000,15.0000,20.0000,40.0000
oasis,1301.0,37.843966,9.912514,6.0000,31.0000,38.0000,45.0000,66.0000
ninety_expire_flag,1301.0,0.509608,0.500100,0.0000,0.0000,1.0000,1.0000,1.0000


## Step 4: Sequence Feature Extraction Functions
Functions to extract and form __single sample tensor__. We begin by writing two functions: one for extracting the vital signs and another for laboratory data. Because of differences in sampling frequencies, we will extract these two feature sequences separetly, and then we will merge them when generating a tensor.

In [5]:
def get_vitals(icustay_id, con, query_schema):
    '''
    Function arguments:
    icustay_id - Numeric ID corresponding to specific ICU stay
    con - SQL connection to MIMIC-III database
    query_schema: string - part of query to tell where to search
    ref_time - if given, used as the reference time for events; 
                otherwise use earliest time in data.
    Returns:
    vitals - DataFrame with patients vitals ordered by time
    '''
    
    # perform SQL pull
    query = query_schema + '''
    SELECT itemid, charttime, icustay_id, valuenum
    FROM vitals
    WHERE icustay_id = {}
    ORDER BY charttime;
    '''.format(icustay_id)
    vitals = pd.read_sql_query(query, con)
    
    vitals_map = {211 : 'HR', 220045: 'HR', 51: 'SBP', 442 : 'SBP',
                  455 : 'SBP', 6701: 'SBP', 220179 : 'SBP', 
                  220050: 'SBP', 8368 : 'DBP', 8440 : 'DBP',
                  8441 : 'DBP', 8555 : 'DBP', 220180 : 'DBP',
                  220051 : 'DBP', 456 : 'MAP', 52 : 'MAP', 
                  6702 : 'MAP', 443 : 'MAP', 220052 : 'MAP',
                  220181 : 'MAP', 225312 : 'MAP', 618 : 'RR',
                  615 : 'RR', 220210 : 'RR', 224690 : 'RR',
                  646 : 'spO2', 220277 : 'spO2', 223762 : 'tempC',
                  676 : 'tempC', 223761 : 'tempF', 678 : 'tempF'}
    vitals['vital_type'] = vitals['itemid'].map(vitals_map)
    del vitals['itemid']
    
    return vitals

def get_labs(icustay_id, con, query_schema):
    '''
    Function arguments:
    icustay_id - Numeric ID corresponding to specific ICU stay
    con - SQL connection to MIMIC-III database
    query_schema: string - part of query to tell where to search
    ref_time - if given, used as the reference time for events; 
                otherwise use earliest time in data.
    Returns:
    labs - DataFrame with patients labs ordered by time
    '''
    
    # perform SQL pull
    query = query_schema + '''
    SELECT label, charttime, icustay_id, valuenum
    FROM lab_item_timeseries
    WHERE icustay_id = {}
    ORDER BY charttime;
    '''.format(icustay_id)
    labs = pd.read_sql_query(query, con)
    
    return labs

We then want to reshape and resample the vitals into 60 minute intervals.

In [6]:
def resample_vitals(vitals):
    '''
    Function arguments:
    vitals - DataFrame with patients vitals ordered by time
    
    Returns:
    vitals - DataFrame with patients vitals ordered by time, 
             resampled down to 60 minutes
    '''
    if (vitals.empty == False):
        try:
            vitals = vitals.pivot_table(index = 'charttime', 
                                        columns = 'vital_type', 
                                        values = 'valuenum', 
                                        aggfunc = 'mean')
            vitals = vitals.resample('60min').mean().reset_index().ffill()
        except:
            return vitals
    
    return vitals

def resample_labs(labs):
    '''
    Function arguments:
    labs - DataFrame with patient labs ordered by time
    
    Returns:
    labs - DataFrame with patient labs ordered by time, 
             resampled down to 60 minutes
    '''
    if (labs.empty == False):
        try:
            # should explore why some indices are repeated; 
            # eric: maybe because multiple concepts occurring same time (two MAP reads)
            labs = labs.pivot_table(index = 'charttime', columns = 'label', values = 'valuenum', aggfunc = 'mean')
            labs = labs.resample('60min').mean().reset_index().ffill()
        except:
            return labs
    
    return labs

Next, we need to convert these data into a tensor that can later be stacked.

In [7]:
def gen_sequence_feature_tensor(vitals, labs, ref_time, cutoff = 24):
    '''
    Function arguments:
    vitals - DataFrame with patient vitals
    labs - DataFrame with patient labs
    ref_time - Reference time to take times relative to
    
    Returns:
    labs_tensor - Numpy array, 3D tensor, corresponding to
                    one sample, feature x time tensor
    '''
    
    features = vitals.merge(labs, on = 'charttime', how = 'left').ffill()
    
    # todo: write a func to clean up temp and then add temp here
    # todo: also clean up this ugly line if possible
    
    try:
        t = np.array(np.array(ref_time) - np.array(features.charttime), dtype = 'timedelta64[h]').astype(float)
    except:
        return np.zeros([1, 26, 1])
    
    
    # vitals
    if ('HR' in features.columns):
        HR = np.array(features.HR)
    else:
        HR = np.ones(len(t))*np.nan
    if ('RR' in features.columns):
        RR = np.array(features.RR)
    else:
        RR = np.ones(len(t))*np.nan
    if ('spO2' in features.columns):
        spO2 = np.array(features.spO2)
    else:
        spO2 = np.ones(len(t))*np.nan
    if ('MAP' in features.columns):
        MAP = np.array(features.MAP)
    else:
        MAP = np.ones(len(t))*np.nan
    if ('SBP' in features.columns):
        SBP = np.array(features.SBP)
    else:
        SBP = np.ones(len(t))*np.nan
    if ('DBP' in features.columns):
        DBP = np.array(features.DBP)
    else:
        DBP = np.ones(len(t))*np.nan
        
    # labs
    if ('BICARBONATE' in features.columns):
        bicarb = np.array(features.BICARBONATE)
    else:
        bicarb = np.ones(len(t))*np.nan
    if ('CREATININE' in features.columns):
        creatinine = np.array(features.CREATININE)
    else:
        creatinine = np.ones(len(t))*np.nan
    if ('CHLORIDE' in features.columns):
        chloride = np.array(features.CHLORIDE)
    else:
        chloride = np.ones(len(t))*np.nan
    if ('GLUCOSE' in features.columns):
        glucose = np.array(features.GLUCOSE)
    else:
        glucose = np.ones(len(t))*np.nan
    if ('HEMATOCRIT' in features.columns):
        hct = np.array(features.HEMATOCRIT)
    else:
        hct = np.ones(len(t))*np.nan
    if ('HEMOGLOBIN' in features.columns):
        hgb = np.array(features.HEMOGLOBIN)
    else:
        hgb = np.ones(len(t))*np.nan
    if ('PLATELET' in features.columns):
        platelet = np.array(features.PLATELET)
    else:
        platelet = np.ones(len(t))*np.nan
    if ('POTASSIUM' in features.columns):
        potassium = np.array(features.POTASSIUM)
    else:
        potassium = np.ones(len(t))*np.nan
    if ('SODIUM' in features.columns):
        sodium = np.array(features.SODIUM)
    else:
        sodium = np.ones(len(t))*np.nan
    if ('BUN' in features.columns):
        bun = np.array(features.BUN)
    else:
        bun = np.ones(len(t))*np.nan
    if ('WBC' in features.columns):
        wbc = np.array(features.WBC)
    else:
        wbc = np.ones(len(t))*np.nan
    
    feature_tensor = np.array([[t, HR, RR, spO2, MAP, SBP, DBP, bicarb, creatinine,\
                                chloride, glucose, hct, hgb, platelet,\
                                potassium, sodium, bun, wbc]])
    return feature_tensor

## Step 5: Format Tensors for RNN
_Considerations:_ Because everyone has a different number of recordings we have to decide how we will handle varying sequence length. How this is handled is crucial because it informs if we'll add padding, where we should cut off recordings, and etc.

We'll begin by defining a function to truncuate at a specific time cutoff.

In [8]:
def trunc_tensor(features, trunc_time = 24):
    '''
    Function arguments:
    vitals - DataFrame with patients vitals
    trunc_time - Time cutoff to truncate recordings at
    
    Returns:
    vitals - Numpy array, 3D tensor, corresponding to
             one sample, feature x time tensor truncated
             at truncation time given by user
    '''
    
    keep = features[0, 0, :] <= trunc_time
    return features[:, :, keep]

We then define a function for adding in the missing rows.

In [9]:
def add_missing_steps(features, missing_indicator = np.nan):
    '''
    Function arguments:
    features - DataFrame with patients features
    
    Returns:
    features - Numpy array, 3D tensor, corresponding to
             one sample, feature x time tensor with
             times missing between t_min and t_max
             filled in with np.nan
    '''
    
    features_tn = np.copy(features)
    try:
        steps_expected = np.max(features_tn[0, 0, :]).astype(int) + 1
    except:
        return features
    
    expected_range = np.array(list(reversed(range(steps_expected))))
    missing_steps = [x for x in expected_range if not x in features_tn[0, 0, :]]
    
    if not missing_steps:
        return features
    
    for i, step in enumerate(missing_steps):
        index = np.where(expected_range == step)[0] 
        empty = np.ones((1, features.shape[1], 1))*missing_indicator
        empty[0, 0, 0] = step
        if (index >= expected_range[0]):
            features_tn = np.append(features_tn, empty, 2)
        else:
            features_tn = np.insert(features_tn, index, empty, 2)
    
    return features_tn

Next, we need a function to pad the tensor with 0s for events which have not yet occurred.

In [10]:
def pad_tensor(features, steps_expected = 24):
    '''
    Function arguents:
    features - a (sorted) Numpy array, 3D tensor, corresponding to
             one sample, feature x time tensor
    '''

    try:
        pad_size = steps_expected - np.max(features[0, 0, :])
    except:
        return features
    
    if (pad_size <= 0):
        return features
    else:
        features_tn = np.copy(features)
        for i in range(pad_size.astype(int)):
            features_tn = np.insert(features_tn, 0, 0, 2)
            
    return features_tn

Finally we wrap all of these functions in one function for feature extraction.

In [11]:
def extract_sequence_features(icustay_id, con, query_schema, ref_time, trunc_time = 24):
    '''
    Function arguments:
    icustay_id - id of the stay of interest
    con - SQL connection to use
    query_schema - MIMIC-III db schema
    ref_time - time to take charttimes relative to
    max_time - t
    
    Returns:
    feature_tensor - Numpy array, 3D tensor, corresponding to
             one sample, feature x time tensor truncated,
             cleaned, and padded
    '''
    
    # initial pull
    vitals_df = get_vitals(icustay_id, con, query_schema)
    labs_df = get_labs(icustay_id, con, query_schema)
  
    # check if pull is empty
    if 'charttime' not in vitals_df.columns or vitals_df.charttime.size == 0:
        return np.nan
    if 'charttime' not in labs_df.columns or labs_df.charttime.size == 0:
        return np.nan
    
    # process DataFrames into a tensor
    vitals_df = resample_vitals(vitals_df)
    labs_df = resample_labs(labs_df)
    
    feature_tensor = gen_sequence_feature_tensor(vitals_df, labs_df, ref_time)
    feature_tensor = trunc_tensor(feature_tensor, trunc_time)
    feature_tensor = add_missing_steps(feature_tensor)
    feature_tensor = pad_tensor(feature_tensor)
    
    return feature_tensor

## Step 6: Cohort-wide Sequence Extraction

We now extract a sequence features tensor for each patient in the cohort.

In [12]:
cohort_ids = icu_cohort.icustay_id

feature_dict = {}

for current_id in cohort_ids:
    current_features = extract_sequence_features(current_id, con, query_schema, icu_cohort.outtime[icu_cohort.icustay_id == current_id], 24)
    if current_features is not np.nan and current_features.shape == (1, 18, 25):
        feature_dict[current_id] = current_features

ICU stays that resulted in a feature tensor returning `np.nan` or with size differing from `(1, 26, 25)` after processing were not in the results, and should be excluded from the cohort. These episodes should be explored further, but there are only ~400 suggesting the issue is not with the code and that these are edge cases. Quick perusal suggests theses are epsiodes in which there were no recordings in the first 24 hours of the stay. While this must be confirmed for all cases, if it is true these patients would be excluded anyway.

In [13]:
missing = list(set(cohort_ids) - set(feature_dict.keys()))
icu_cohort = icu_cohort[icu_cohort.icustay_id.isin(missing) == False]

We then stack the resulting tensors.

In [14]:
features_sequence = np.vstack(feature_dict.values())[:, 1:26, :]

As a check, we compare the number of samples in each frame.

In [15]:
features_sequence.shape[0] == icu_cohort.shape[0]

True

## Step 7: Static Feature Tensor Formatting

We now set up vectors for the non-sequence input.

__Gender:__ binarized (only M or F)

In [19]:
# Convert to vector (label encode)
map_gender, f_gender = np.unique(icu_cohort.gender, return_inverse=True)
print(map_gender)
f_gender.shape

['F' 'M']


(1284,)

__Admission age:__ continuous variable mapped piecewise

In [20]:
f_admission_age = np.copy(icu_cohort.admission_age)
f_admission_age[(f_admission_age > 89.5)] = 91.4
map_admission_age = np.copy(f_admission_age) # for symmetry

__Ethnicity:__ one hot encoded

In [23]:
le = LabelEncoder()
f_ethnicity = le.fit_transform(icu_cohort.ethnicity)
map_ethnicity = le.classes_

__LoS in ICU:__ continuous variable

*Setting all nans to the minimum value for now , need to figure out why they exist at all*

In [24]:
f_los_icu = np.copy(icu_cohort.los_icu)
f_los_icu[np.isnan(icu_cohort.los_icu)] = np.min(icu_cohort.los_icu)
map_los_icu = np.copy(f_los_icu) # for symmetry

__Oasis and Elixhauser:__ Integers

In [25]:
f_oasis = icu_cohort.oasis
map_oasis = np.copy(f_oasis)
f_elixhauser_vanwalraven = icu_cohort.elixhauser_vanwalraven
map_elixhauser_vanwalraven = np.copy(f_elixhauser_vanwalraven)

Then save these to a tensor.

In [26]:
features_static = np.array([f_gender,
     f_admission_age,
     f_ethnicity,
     f_los_icu,
     f_oasis,
     f_elixhauser_vanwalraven]).T

## Step 8: Save Tensors

All numpy files saved to `./data` directory.

In [30]:
data_dir = './'
import os
# Save static features
np.save(data_dir + os.path.sep + 'features_sequence.npy', features_sequence)
# Save static features
np.save(data_dir + os.path.sep + 'features_static.npy', features_static)
# Save Label
label = icu_cohort.ninety_expire_flag
np.save(data_dir + os.path.sep + 'labels.npy', label)